# Label Inference

In [1]:
import numpy as np
import torch
from torchvision import transforms, datasets
from torchvision.utils import save_image

import unsplit.attacks as unsplit
from unsplit.models import *
from unsplit.util import *

Change the dataset and count values as desired. Dataset can be one of `mnist`, `f_mnist`, or `cifar`. The `count` variable denotes how many examples the attack will run on.

In [8]:
count = 1000

In [5]:
dataset = 'cifar'

if dataset == 'mnist':
    trainset = datasets.MNIST('data/mnist', download=True, train=True, transform=transforms.ToTensor())
    testset = datasets.MNIST('data/mnist', download=True, train=False, transform=transforms.ToTensor())
    client, server, clone = MnistNet(), MnistNet(), MnistNet()
    split_layer = 9
    grad_index = 8
elif dataset == 'f_mnist':
    trainset = datasets.FashionMNIST('data/f_mnist', download=True, train=True, transform=transforms.ToTensor())
    testset = datasets.FashionMNIST('data/f_mnist', download=True, train=False, transform=transforms.ToTensor())
    client, server, clone = MnistNet(), MnistNet(), MnistNet()
    split_layer = 9
    grad_index = 8
elif dataset == 'cifar':
    trainset = datasets.CIFAR10('data/cifar', download=True, train=True, transform=transforms.ToTensor())
    testset = datasets.CIFAR10('data/cifar', download=True, train=False, transform=transforms.ToTensor())
    client, server, clone = CifarNet(), CifarNet(), CifarNet()
    split_layer = 16
    grad_index = 14

trainloader = torch.utils.data.DataLoader(trainset, shuffle=True, batch_size=64)
testloader = torch.utils.data.DataLoader(testset, shuffle=True)

100%|███████████████████████| 170498071/170498071 [00:10<00:00, 15667488.64it/s]


Extracting data/cifar/cifar-10-python.tar.gz to data/cifar
Files already downloaded and verified


To launch the attack, we first initialize the necessary variables.

In [6]:
client_opt = torch.optim.Adam(client.parameters(), lr=0.001, amsgrad=True)
server_opt = torch.optim.Adam(server.parameters(), lr=0.001, amsgrad=True)
clone_opt = torch.optim.Adam(clone.parameters(), lr=0.001, amsgrad=True)

criterion = torch.nn.CrossEntropyLoss()

The following cell runs the attack, iterating over the examples. Comments in the code explain the attack in more detail.

In [ ]:
results = []
for idx, (image, label) in enumerate(testloader):
    if idx == count:
        break

    # enumerate possible label values
    label_vals = [i * torch.ones(len(label)).long() for i in range(10)] 

    # obtain gradient values from client
    client_opt.zero_grad()
    server_opt.zero_grad()
    server_out = server(image, end=split_layer)
    pred = client(server_out, start=split_layer+1)
    loss = criterion(pred, label)
    loss.backward(retain_graph=True)

    target_grad = [param.grad for param in client.parameters()][grad_index]

    # obtain clone model's output
    clone_opt.zero_grad()
    clone_pred = clone(server_out, start=split_layer+1)

    # try out all possible labels and pick the one that produces the closest gradient values
    pred_label = unsplit.label_inference(clone_pred, clone, target_grad, label_vals, grad_index)

    results.append(label.item() == pred_label.item())
    print(f'Label: {label.item()} - Predicted: {pred_label.item()}')

    # perform training updates
    clone_loss = criterion(clone_pred, pred_label)
    clone_loss.backward()
    client_opt.step()
    clone_opt.step()
    server_opt.step()

print('Run complete.')
print(f'Label inference accuracy: {sum(results) / count}')